In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
import os

In [7]:
# Commodity symbols and their respective Yahoo Finance codes
commodities = {
    "silver": "SI=F",
    "bac": "BAC",
    "citi": "C",
    "corn": "ZC=F",
    "euro": "EURUSD=X",
    "gold": "GC=F",
    "oil": "CL=F",
    "pound": "GBPUSD=X",
    "soybns": "ZS=F",
    "tr5yr": "^FVX",
    "tr10yr": "^TNX",
    "wheat": "ZW=F",
    "yen": "JPY=X"
}

# Function to append text to a file
def save_to_file(file_name, text):
    with open(file_name, 'a') as f:
        f.write(text + '\n')

# Ensure the directories for Excel files and images exist
excel_dir = './content/excel_files'
metrics_dir = './content/metrics'

if not os.path.exists(metrics_dir):
    os.makedirs(metrics_dir)

if not os.path.exists(excel_dir):
    os.makedirs(excel_dir)


In [8]:
# Define functions to calculate match ratio and correlation for "up" and "down" reversal conditions
def calculate_metrics(bac_data_new,reversal_condition, indicator_name, direction):
  bac_data_new[indicator_name + '_reversal'] = reversal_condition
  mbp_reversal_dates = bac_data_new[bac_data_new[f'reversal_mbp{direction}']].index
  indicator_reversal_dates = bac_data_new[bac_data_new[indicator_name + '_reversal']].index
  time_window = 5
  matched_count = sum(any(bac_data_new.loc[(bac_data_new.index >= mbp_date - pd.Timedelta(days=time_window)) &
                                            (bac_data_new.index <= mbp_date + pd.Timedelta(days=time_window)),
                                            indicator_name + '_reversal']) for mbp_date in mbp_reversal_dates)
  match_ratio = matched_count / len(mbp_reversal_dates)
  date_range = pd.date_range(start=bac_data_new.index.min(), end=bac_data_new.index.max())
  daily_mbp = pd.Series(0, index=date_range)
  daily_mbp[mbp_reversal_dates] = 1
  daily_indicator = pd.Series(0, index=date_range)
  daily_indicator[indicator_reversal_dates] = 1
  correlation = daily_mbp.corr(daily_indicator)
  return match_ratio, correlation

In [9]:
# Main processing function for each commodity
def process_commodity(commodity_name, symbol):
  # Load the Excel file
  df = pd.read_excel(f"./content/{commodity_name}.xlsx")
  df = df.drop('maturity_target', axis=1)
  images_dir = f'./content/images/{commodity_name}'
  if not os.path.exists(images_dir):
    os.makedirs(images_dir)

  # Assuming prInc and prDec are columns in the same sheet as idt
  # Calculate labels for Market-Based Probabilities
  df['label_mbp'] = df.apply(lambda row: -1 if row['prDec'] > row['prInc'] else 1, axis=1)

  # Identify reversal points for Market-Based Probabilities
  df['diff_label_mbp'] = df['label_mbp'].diff()
  #up
  df['reversal_mbpup'] = df['diff_label_mbp'] == 2
  #down
  df['reversal_mbpdown'] = df['diff_label_mbp'] == -2

  # Get the 'idt' column as 'date' and convert it to datetime
  df['date'] = pd.to_datetime(df['idt'])
  # Ensure the date range includes enough days before the start date to calculate the moving average
  start_date = df['date'].min() - pd.Timedelta(days=30)  # Increased padding to account for non-trading days
  end_date = df['date'].max() + pd.Timedelta(days=30)
  df.set_index('date', inplace=True)

  # Fetch real financial data
  stock_data = yf.download(symbol, start=start_date, end=end_date)

  # Reset index to merge on date
  stock_data.reset_index(inplace=True)

  # Perform a left merge
  bac_data_new = pd.merge(df, stock_data, how='left', left_index=True, right_on='Date')
  # Prepare the data with indicators
  bac_data_new['Date'] = pd.to_datetime(bac_data_new['Date'])
  bac_data_new['idt'] = pd.to_datetime(bac_data_new['idt'])
  bac_data_new.set_index('Date', inplace=True)
  bac_data_new['MA20'] = bac_data_new['Close'].rolling(window=20).mean()
  bac_data_new['MA50'] = bac_data_new['Close'].rolling(window=50).mean()
  bac_data_new['20dSTD'] = bac_data_new['Close'].rolling(window=20).std()
  bac_data_new['Upper_Band'] = bac_data_new['MA20'] + (bac_data_new['20dSTD'] * 2)
  bac_data_new['Lower_Band'] = bac_data_new['MA20'] - (bac_data_new['20dSTD'] * 2)
  bac_data_new['EMA12'] = bac_data_new['Close'].ewm(span=12, adjust=False).mean()
  bac_data_new['EMA26'] = bac_data_new['Close'].ewm(span=26, adjust=False).mean()
  bac_data_new['MACD'] = bac_data_new['EMA12'] - bac_data_new['EMA26']
  bac_data_new['Signal_Line'] = bac_data_new['MACD'].ewm(span=9, adjust=False).mean()
  low_min = bac_data_new['Low'].rolling(window=14).min()
  high_max = bac_data_new['High'].rolling(window=14).max()
  bac_data_new['Stochastic'] = 100 * ((bac_data_new['Close'] - low_min) / (high_max - low_min))
  delta = bac_data_new['Close'].diff(1)
  gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
  loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
  rs = gain / loss
  bac_data_new['RSI'] = 100 - (100 / (1 + rs))

  bac_data_new.dropna(inplace=True)

  # Split the data by year and plot each year's data separately
  years = bac_data_new.index.year.unique()  # Get unique years

  # Initialize lists to store dataframes for Excel export
  dfs_up = []
  dfs_down = []

  # Process data by year
  for year in years:
      df_year = bac_data_new[bac_data_new.index.year == year]
      crossover_up_year = (df_year['MA20'] > df_year['MA50']) & (df_year['MA20'].shift(1) < df_year['MA50'].shift(1))
      crossover_down_year = (df_year['MA20'] < df_year['MA50']) & (df_year['MA20'].shift(1) > df_year['MA50'].shift(1))
      touch_upper_year = df_year['Close'] > df_year['Upper_Band']
      touch_lower_year = df_year['Close'] < df_year['Lower_Band']
      reversalup_year = df_year['reversal_mbpup']
      reversaldown_year = df_year['reversal_mbpdown']

      # Create dataframes for Excel export
      df_up = pd.DataFrame({
          'Commodity': commodity_name,
          'Date': df_year.index,
          'MA Crossover Up': crossover_up_year.astype(int),
          'Touch Lower Band': touch_lower_year.astype(int),
          'Fed Reversal Pointsup': reversalup_year.astype(int)
      })

      df_down = pd.DataFrame({
          'Commodity': commodity_name,
          'Date': df_year.index,
          'MA Crossover Down': crossover_down_year.astype(int),
          'Touch Upper Band': touch_upper_year.astype(int),
          'Fed Reversal Pointsdown': reversaldown_year.astype(int)
      })

      dfs_up.append(df_up)
      dfs_down.append(df_down)

      # Plotting for each year
      plt.figure(figsize=(14,7))

      # Closing Price
      plt.plot(df_year.index, df_year['Close'], label='Close Price', color='blue')

      # Moving Averages
      plt.plot(df_year.index, df_year['MA20'], label='20-day MA', color='red')
      plt.plot(df_year.index, df_year['MA50'], label='50-day MA', color='green')

      # Bollinger Bands
      plt.plot(df_year.index, df_year['Upper_Band'], label='Upper Bollinger Band', color='cyan')
      plt.plot(df_year.index, df_year['Lower_Band'], label='Lower Bollinger Band', color='cyan', alpha=0.3)

      # Highlight Reversal Points
      plt.scatter(df_year.index[crossover_up_year], df_year['Close'][crossover_up_year], color='magenta', label='MA Crossover Up', marker='^', s=100)
      plt.scatter(df_year.index[crossover_down_year], df_year['Close'][crossover_down_year], color='black', label='MA Crossover Down', marker='v', s=100)
      plt.scatter(df_year.index[touch_upper_year], df_year['Close'][touch_upper_year], color='orange', label='Touch Upper Band', marker='o', s=50)
      plt.scatter(df_year.index[touch_lower_year], df_year['Close'][touch_lower_year], color='purple', label='Touch Lower Band', marker='o', s=50)

      # Highlighting Reversal Points up and down
      if reversalup_year.any():
          plt.scatter(df_year.index[reversalup_year], df_year['Close'][reversalup_year], color='red', label='Fed Reversal Pointsup', marker='o', s=100)
      if reversaldown_year.any():
          plt.scatter(df_year.index[reversaldown_year], df_year['Close'][reversaldown_year], color='pink', label='Fed Reversal Pointsdown', marker='o', s=100)

      plt.title(f'{commodity_name} Stock Price with Reversal Points - {year}')
      plt.legend()
      plt.savefig(f'{images_dir}/{commodity_name}_stock_price_{year}.png')  # Save figure to images directory
      plt.close()

  # Concatenate dataframes and save to Excel
  df_up_concat = pd.concat(dfs_up)
  df_down_concat = pd.concat(dfs_down)
  df_up_concat.to_excel(f'{excel_dir}/{commodity_name}_up_events.xlsx', index=False)
  df_down_concat.to_excel(f'{excel_dir}/{commodity_name}_down_events.xlsx', index=False)
  # Ensure any existing metrics file is cleared

  # Initialize a dictionary to hold the results
  results = {}

  # Calculate metrics for each technical indicator for "up" reversals
  results['MA_Crossoverup'] =calculate_metrics(bac_data_new,(bac_data_new['MA20'] > bac_data_new['MA50']) & (bac_data_new['MA20'].shift(1) < bac_data_new['MA50'].shift(1)), 'MA_Crossover_Up', 'up')
  results['Bollinger_Bandsup'] =calculate_metrics(bac_data_new,bac_data_new['Close'] > bac_data_new['Upper_Band'], 'Bollinger_Bands_Up',  'up')
  results['MACDup'] = calculate_metrics(bac_data_new,bac_data_new['MACD'] > bac_data_new['Signal_Line'], 'MACD_Up',  'up')
  results['RSIup'] = calculate_metrics(bac_data_new,bac_data_new['RSI'] < 30, 'RSI_Up',  'up')

  # Calculate metrics for each technical indicator for "down" reversals
  results['MA_Crossoverdown'] =calculate_metrics(bac_data_new,(bac_data_new['MA20'] < bac_data_new['MA50']) & (bac_data_new['MA20'].shift(1) > bac_data_new['MA50'].shift(1)), 'MA_Crossover_Down','down')
  results['Bollinger_Bandsdown'] = calculate_metrics(bac_data_new,bac_data_new['Close'] < bac_data_new['Lower_Band'], 'Bollinger_Bands_Down', 'down')
  results['MACDdown'] =calculate_metrics(bac_data_new,bac_data_new['MACD'] < bac_data_new['Signal_Line'], 'MACD_Down',  'down')
  results['RSIdown'] = calculate_metrics(bac_data_new,bac_data_new['RSI'] > 70, 'RSI_Down','down')

  # Convert results dictionary to a DataFrame
  metrics_df = pd.DataFrame(results, index=['Matching ratio', 'Correlation coefficient']).T

  # Save the DataFrame to an Excel file
  metrics_df.to_excel(f'{metrics_dir}/{commodity_name}technical_indicators_metrics.xlsx')

  print(f"The metrics have been calculated and saved to {commodity_name}technical_indicators_metrics.xlsx.")

In [10]:
for key, value in commodities.items():
  process_commodity(key, value)

[*********************100%%**********************]  1 of 1 completed


The metrics have been calculated and saved to silvertechnical_indicators_metrics.xlsx.


[*********************100%%**********************]  1 of 1 completed


The metrics have been calculated and saved to bactechnical_indicators_metrics.xlsx.


[*********************100%%**********************]  1 of 1 completed


...

The metrics have been calculated and saved to tr10yrtechnical_indicators_metrics.xlsx.


[*********************100%%**********************]  1 of 1 completed


The metrics have been calculated and saved to wheattechnical_indicators_metrics.xlsx.


[*********************100%%**********************]  1 of 1 completed


The metrics have been calculated and saved to yentechnical_indicators_metrics.xlsx.
